# Word Embedding

This lesson is designed to explore features of word embeddings described by Ben Schmidt in his blog post <a href="http://bookworm.benschmidt.org/posts/2015-10-30-rejecting-the-gender-binary.html">"Rejecting the Gender Binary"</a>.

The primary corpus we use consists of the <a href="http://txtlab.org/?p=601">150 English-language novels</a> made available by the .txtLab at McGill. We also look at a <a href="http://ryanheuser.org/word-vectors-1/">Word2Vec model trained on the ECCO-TCP corpus</a> of 2,350 eighteenth-century literary texts made available by Ryan Heuser. (I have shortened the number of terms in the model by half in order to conserve memory.)

For background on Word2Vec's mechanics, I suggest this <a href="https://www.tensorflow.org/versions/r0.8/tutorials/word2vec/index.html">brief tutorial</a> by Google, especially the sections "Motivation," "Skip-Gram Model," and "Visualizing."

### Lesson
<ol>
<li>Pre-Processing</li>
<li>Word2Vec</li>
<ol><li>Training</li>
<li>Embeddings</li>
<li>Visualization</li>
</ol>
<li>Saving/Loading Models</li>
<li>Word2Vec from Scratch</li>
<li>Appendix: Vector Rejection</li>
</ol>

# 0. Prep

In [ ]:
# Visualization
%pylab inline
matplotlib.style.use('ggplot')

# Data Wrangling
from datascience import *
import numpy as np
from scipy.spatial.distance import cosine

# Note: 'datascience' is a custom package used by Berkeley's
# Foundations of Data Science course. In practice it is very
# similar to the more-standard 'pandas'

# Natural Language Processing
import gensim
import nltk
nltk.download('punkt')

### Corpus Description
English-language subset of Andrew Piper's novel corpus, totaling 150 novels by British and American authors spanning the years 1771-1930. These texts reside on disk, each in a separate plaintext file. Metadata is contained in a spreadsheet distributed with the novel files.

### Metadata Columns
<ol><li>Filename: Name of file on disk</li>
<li>ID: Unique ID in Piper corpus</li>
<li>Language: Language of novel</li>
<li>Date: Initial publication date</li>
<li>Title: Title of novel</li>
<li>Gender: Authorial gender</li>
<li>Person: Textual perspective</li>
<li>Length: Number of tokens in novel</li></ol>

In [ ]:
# Import Metadata
metadata_tb = Table.read_table('txtlab_Novel150_English.csv')


# Alternately, using pandas:

# import pandas
# metadata_tb = pandas.read_csv('txtlab_Novel150_English.csv')

In [ ]:
# Set location of corpus folder
fiction_path = 'txtalb_Novel150_English/'

In [ ]:
# Create empty list, entries will be list of tokens from each novel
novel_list = []

# Iterate through filenames in metadata table
for filename in metadata_tb['filename']:
    
    # Read in novel text as single string, make lowercase
    with open(fiction_path+filename, 'r') as file_in:
        novel = file_in.read()
    
    # Add novel text as single string to master list
    novel_list.append(novel)

# 1. Pre-Processing

Word2Vec learns about the relationships among words by observing them in context. We'll need to tokenize the words in our corpus while retaining sentence boundaries. Since novels were imported as single strings, we'll first use <i>sent_tokenize</i> to divide them into sentences, and second, we'll split each sentence into its own list of words.

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
# Even though I love nltk's word_tokenize(), I just need a
# quick and dirty tokenizer. Emphasis on quick.

def fast_tokenize(text):
    
    # Get a list of punctuation marks
    from string import punctuation
    
    # Iterate through text removing punctuation characters
    no_punct = "".join([char for char in text if char not in punctuation])
    
    # Split text over whitespace into list of words
    tokens = no_punct.split()
    
    return tokens

In [ ]:
sentences = [sentence for novel in novel_list for sentence in sent_tokenize(novel)]

In [ ]:
words_by_sentence = [fast_tokenize(sentence.lower()) for sentence in sentences]

In [ ]:
# Since fast_tokenize() is a little clunky, we'll explicitly remove
# any sentences that contain zero tokens
words_by_sentence = [sentence for sentence in words_by_sentence if sentence != []]

In [ ]:
# Inspect
words_by_sentence[0]

# 2. Word2Vec

### Word Embedding
Word2Vec is the most prominent word embedding algorithm. Word embedding generally attempts to identify semantic relationships between words by observing them in context.

Imagine that each word in a novel has its meaning determined by the ones that surround it in a limited window. For example, in Moby Dick's first sentence, “me” is paired on either side by “Call” and “Ishmael.” After observing the windows around every word in the novel (or many novels), the computer will notice a pattern in which “me” falls between similar pairs of words to “her,” “him,” or “them.” Of course, the computer had gone through a similar process over the words “Call” and “Ishmael,” for which “me” is reciprocally part of their contexts.  This chaining of signifiers to one another mirrors some of humanists' most sophisticated interpretative frameworks of language.

The two main flavors of Word2Vec are CBOW (Continuous Bag of Words) and Skip-Gram, which can be distinguished partly by their input and output during training. Skip-Gram takes a word of interest as its input (e.g. "me") and tries to learn how to predict its context words ("Call","Ishmael"). CBOW does the opposite, taking the context words ("Call","Ishmael") as a single input and tries to predict the word of interest ("me").

In general, CBOW is is faster and does well with frequent words, while Skip-Gram potentially represents rare words better.

### Word2Vec Features
<ul>
<li>Size: Number of dimensions for word embedding model</li>
<li>Window: Number of context words to observe in each direction</li>
<li>min_count: Minimum frequency for words included in model</li>
<li>sg (Skip-Gram): '0' indicates CBOW model; '1' indicates Skip-Gram</li>
<li>Alpha: Learning rate (initial); prevents model from over-correcting, enables finer tuning</li>
<li>Iterations: Number of passes through dataset</li>
<li>Batch Size: Number of words to sample from data during each pass</li>
</ul>

Note: Script uses default value for each argument

## Training

In [ ]:
# Train word2vec using CBOW
model = gensim.models.Word2Vec(words_by_sentence, size=100, window=5, \
                               min_count=5, sg=0, alpha=0.025, iter=5, batch_words=10000)

## Embeddings

In [ ]:
# Return dense word vector
model['whale']

In [ ]:
# Find cosine distance between two word vectors
model.similarity('sense','sensibility')

In [ ]:
# Find cosine distance between two clusters of word vectors
# Each cluster is measured as the mean of its words

model.n_similarity(['sense','sensibility'],['whale','harpoon'])

In [ ]:
# Finds mean vector of words in list
# and identifies the word further from that mean

model.doesnt_match(['pride','prejudice', 'harpoon'])

In [ ]:
# The canonic word2vec exercise: King - Man + Woman -> Queen
model.most_similar(positive=['woman', 'king'], negative=['man'])

In [ ]:
# Can we find gender a la Schmidt?
# Note that this method uses projection, whereas Schmidt had used rejection

model.most_similar(positive=['she','her','hers','herself'], negative=['he','him','his','himself'])

In [ ]:
model.most_similar(positive=['he','him','his','himself'], negative=['she','her','hers','herself'])

In [ ]:
model.most_similar(positive=['she','her','hers','herself','he','him','his','himself'], topn=50)

In [ ]:
## EX. Use the most_similar method to find the tokens nearest to 'car' in our model.
##     Do the same for 'motorcar'.

## Q.  What characterizes each word in our corpus? Does this make sense?


## EX. Vector addition and subtraction can be thought of in terms of analogy.
##     From the example above: 'man' is to 'king' as 'woman' is to '???'
##     Use the most_similar method to find: 'paris' is to 'france' as 'london' is to '???'

## Q.  What has our model learned about nation-states?


## EX. Perform the canonic Word2Vec addition again but leave out a term:
##     Try 'king' - 'man', 'woman' - 'man', 'woman' + 'king'

## Q.  What do these indicate semantically?

## Visualization

In [ ]:
# Dictionary of words in model
model.vocab

In [ ]:
# Visualizing the whole vocabulary would make it hard to read
len(model.vocab)

In [ ]:
# For interpretability, we'll select words that already have a semantic relation

her_tokens = [token for token,weight in model.most_similar(positive=['she','her','hers','herself'], \
                                                       negative=['he','him','his','himself'], topn=50)]

In [ ]:
# Get the vector for each sampled word
vectors = [model[word] for word in her_tokens]    

In [ ]:
# Calculate distances among texts in vector space

from sklearn.metrics import pairwise
dist_matrix = pairwise.pairwise_distances(vectors, metric='cosine')

In [ ]:
# Multi-Dimensional Scaling

from sklearn.manifold import MDS
mds = MDS(n_components = 2, dissimilarity='precomputed')
embeddings = mds.fit_transform(dist_matrix)

In [ ]:
# Fussing with matplotlib

_, ax = plt.subplots(figsize=(10,10))
ax.scatter(embeddings[:,0], embeddings[:,1], alpha=0)
for i in range(len(vectors)):
    ax.annotate(her_tokens[i], ((embeddings[i,0], embeddings[i,1])))

In [ ]:
## Q. What kinds of semantic relationships exist in the diagram above?
##    Are there any words that seem out of place?

# 3. Saving/Loading Models

In [ ]:
# Save current model for later use
model.save_word2vec_format('word2vec.txtalb_Novel150_English.txt')

In [ ]:
# Load up models from disk

# Model trained on Eighteenth Century Collections Online corpus (~2500 texts)
# Made available by Ryan Heuser: http://ryanheuser.org/word-vectors-1/

ecco_model = gensim.models.Word2Vec.load_word2vec_format('word2vec.ECCO-TCP.txt')

In [ ]:
# Model trained on Reuters news articles
reuters_model = gensim.models.Word2Vec.load_word2vec_format('word2vec.reuters.txt')

In [ ]:
## EX. Heuser's blog post explores an analogy in eighteenth-century thought that
##     Riches are to Virtue what Learning is to Genius. How true is this in
##     the ECCO-trained Word2Vec model? Is it true in the one we trained?

##  Q. How might we compare word2vec models more generally?

# 4. Word2Vec from Scratch

### Neural Networks
The Word2Vec algorithm is a clever implementation of a more general Neural Network.

Neural Networks can be thought of as <i>function approximators</i>. Oftentimes, we have inputs and outputs that can be represented as vectors but do not know the function that transforms input to output even though we would like to reproduce it. For example, if we throw a ball directly upward, we already know based on Newtonian physics that its height will be a function of time as determined by gravity and the initial velocity, and that we can identify its path in terms of these. However, without knowing anything about these features of motion, a Neural Network could be used to reverse engineer the path of the ball by showing it a series of inputs (times) and their associated outputs (heights). In turn, the Neural Network would be able to predict the ball's height at later times or in between the times it had been shown.

Neural Networks are very often used to approximate functions that identify whether images have cats in them.

At the grittiest mathematical level, Neural Networks consist of (at least!) two matrices. For convenience, we'll call them <i>Matrix A</i> and <i>Matrix B</i>. Without going into the Linear Algebra, the input vector is initially multiplied by Matrix A, creating a new vector that is then multiplied by Matrix B, producing the network's output vector. And that's it. The key here are the values contained in each of the matrices.

The values of the network's matrices are learned through an iterative, correction process. Initially, the matrices are assigned random values, so naturally the network's output is random as well. However, that intial output gets compared to a desired, target vector and based on its error, the matrices get tweaked. In our previous example, the network would have been trained by showing it a given time (say, 2 seconds after throwing the ball) and comparing its predicted output (say, a randomly chosen -127 feet above ground) to the target output (say, 11 feet above ground). The values of Matrix B would first need to be revised in order to produce the correct output and the values of Matrix A would be revised based on those of Matrix B. Further pairs of inputs and outputs would be shown to the Network and the values of each matrix further updated.

Once the network has been trained, we might think of Matrix A as encoding relationships among inputs and Matrix B as decoding these into outputs. Word2Vec takes advantage of the fact that when we train a Neural Network on CBOW or Skip-Gram models of language, each line of Matrix A represents a unique word's projection into semantic space.

### A Note About Terminology
I have used my own terms to describe the structure of a Neural Network, since they are simpler for our particular application. Networks are often described in terms of Layers, Nodes, and Weights.

The input vector might be referred to as Layer 0, or the input layer. The vector that is produced by multiplying the input vector by Matrix A is Layer 1, or more often, the hidden layer. And the vector that is produced after multiplication with Matrix B is Layer 2, or the output layer. The network I have described has just two layers, but there may be potentially very many -- i.e. many hidden layers between the input and output. This is referred to as <i>deep learning</i>.

Matrices A and B are typically referred to as Weight Matrices. The number of dimensions in them are referred to as nodes. The idea is that each node in each layer is connected to each node in the next one, akin to the neurons of a human brain. The matrices record the strength of connection between each node.

### One-Hot Vector Representation of Words
In order to make useful inputs and targets for our Neural Network, words initially get represented by a <i>one-hot vector</i>. Essentially, the one-hot encoding of a word is a document-term matrix for a single document with a single word and as many columns as there are unique words in the entire corpus. Every value for the document is zero except in the column for the word in question. This strategy is used to distinguish words from one another when they are represented as inputs and targets. Since the values of the vector are entirely independent of one another, no relationships between words get expressed by these.

For CBOW, the target output is simply the one-hot vector of the word in question. The input is the average of the one-hot vectors representing context words within a given window.

### Word2Vec Features
<ul>
<li>Vocabulary: Most frequent words to include in our vocabulary; determines size of one-hot word vectors and, in turn, one dimensions of  matrices</li>
<li>Semantic Space: Dimensions in which words' relationships get encoded; determines other dimension of matrices; alternately referred to as the number of <i>hidden nodes</i></li>
<li>Window: Context words to observe left and right</li>
<li>Alpha: learning rate; prevents model from over-correcting, enables finer tuning</li>
<li>Epochs: Passes through dataset</li>
<li>Batch Size: Words to sample from data during each pass</li>
</ul>

In [ ]:
# Set value for each feature

vocab_size = 10000
semantic_space_size = 100
window = 5
alpha = 1e-4
epochs = 5
batch_size = 10000

In [ ]:
# Assign each unique token to a column in the one-hot vector

from collections import Counter

all_tokens = [token for sentence in words_by_sentence for token in sentence]
token_counts = Counter(all_tokens)
common_tokens = [token for token,frequency in token_counts.most_common(vocab_size-1)]

# We'll add a dummy term to represent all infrequent words
common_tokens.append('_SLUG_')

token_to_ix = { tk:i for i,tk in enumerate(common_tokens) }
ix_to_token = { i:tk for i,tk in enumerate(common_tokens) }

In [ ]:
# Inspect
token_to_ix['whale']

In [ ]:
# Inspect
ix_to_token[1618]

In [ ]:
# Initialize matrices with random values

Matrix_A = np.random.randn(vocab_size,semantic_space_size)*0.01
Matrix_B = np.random.randn(semantic_space_size,vocab_size)*0.01

In [ ]:
# Inspect
Matrix_A

In [ ]:
# We need two functions:
# 1) Produce appropriate input and output vectors for our CBOW model, given a list of tokens in a sentence
# 2) Update the values in our matrices after each observation

def CBOW_input_output(sentence):
    
    # Get one-hot values for each word in sentence
    ix_map = [token_to_ix[word] if word in common_tokens else token_to_ix['_SLUG_'] for word in sentence]
    
    # Randomly select a target word
    target_index = np.random.choice(range(len(sentence)))
    
    
    # Create (target) vector in which all values are zero
    target_vector = np.zeros((vocab_size,1))
    
    # Set target-word value to 1
    target_vector[ix_map[target_index]] = 1
    
    
    # Create (input) vector in which all values are zero
    input_vector = np.zeros((vocab_size,1))
    
    # Iterate through each of window-distance words to left of target
    for i in range(target_index-window, target_index):
        
        # Add 1 to each word's value in the input vector
        if i in range(len(sentence)) and sentence[i] in common_tokens:
            input_vector[ix_map[i]] += 1

            
    # Iterate through each of window-distance words to left of target
    for i in range(target_index+1, target_index+window+1):
        
        # Add 1 to each word's value in the input vector
        if i in range(len(sentence)) and sentence[i] in common_tokens:
            input_vector[ix_map[i]] += 1
        

    
    # Divide by number of words observed
    input_vector = input_vector/(sum(input_vector)+1e-8)
            
    return input_vector, target_vector



def train_function(input_vector_, target_vector_, Matrix_A_, Matrix_B_, alpha_):
    
    # Multiply input by matrices to produce output
    hidden_vector = np.dot(input_vector_.T,Matrix_A_)
    output_vector = 1/(1+np.exp(-(np.dot(hidden_vector,Matrix_B_))))
    
    # Note that after the output vector is produced, it is put into an activation function
    # -- in this case, the logistic function -- that squashes its values between (0,1).
    # This basically measures each vocabulary word's probability of being the true output.
    
    # Determine error for each matrix
    output_delta = (target_vector_.T - output_vector)
    hidden_delta =  np.dot(output_delta,Matrix_B_.T)

    # Update matrix values
    Matrix_B_ += np.dot(hidden_vector.T,output_delta)*alpha_
    Matrix_A_ += np.dot(input_vector_,hidden_delta)*alpha_
    
    return(Matrix_A_,Matrix_B_)

In [ ]:
# Train our Neural Network

for _ in range(epochs):
    sampled_sentences = np.random.choice(words_by_sentence, batch_size)
    for sentence in sampled_sentences:
        input_vector, target_vector = CBOW_input_output(sentence)
        Matrix_A, Matrix_B = train_function(input_vector, target_vector, Matrix_A, Matrix_B, alpha)

In [ ]:
# Inspect
Matrix_A

In [ ]:
# Normalize each matrix row

row_norm = np.linalg.norm(Matrix_A, axis=1)
Matrix_A_norm = Matrix_A / row_norm[:, numpy.newaxis]

In [ ]:
# Create functions to retrieve word embeddings, determine similarities

def get_vector(word,embedding_matrix=Matrix_A_norm):
    if word in common_tokens:
        ix = token_to_ix[word]
        return embedding_matrix[ix]
    else:
        print(word, "not in vocabulary")
    

def token_similarity(token_1,token_2):
    vector_1 = get_vector(token_1)
    vector_2 = get_vector(token_2)
    return 1-cosine(vector_1,vector_2)

In [ ]:
# Retrieve word embedding
get_vector('whale')

In [ ]:
# Pronouns should be close to one another
token_similarity('she','her')

In [ ]:
# Let's compare a pronoun to a control word with a different POS
token_similarity('call','me')

In [ ]:
# Can we recreate the analogy 'king' - 'man' + 'woman' ~ 'queen?
# This formulation plays with some of the underlying algebra

token_similarity('king','queen') - token_similarity('man','queen') + token_similarity('king','queen')

In [ ]:
## EX. Write a function that will produce inputs and outputs for the Skip-Gram model.

## EX. Our model randomly selects words from the corpus, so it implicitly prefers frequent tokens.
##     Modify the script to minimize (but not eliminate) appearances of high-frequency terms.
##     as components of input vectors.

## EX. It has been found that the learning process can be sped up by showing "wrong" targets
##     during training and penalizing the network when it predicts them as outputs.
##     Implement this.

# Appendix: Vector Rejection

In [ ]:
# Create functions that reject vectors, find nearest words

def vector_rejector(vector_1,vector_2):
    # Rejects vector_2 from vector_1
    # Note that vectors are passed in, not words; also mean of
    # several vectors could be passed as a single argument
    vector_2_norm = vector_2 / np.linalg.norm(vector_2)
    projection = np.dot(vector_1,vector_2_norm) * vector_2_norm
    return vector_1 - projection

def nearest_words(vector,embedding_matrix=Matrix_A_norm):
    # Note that this relies on our 'ix_to_token' dictionary;
    # format of embedding matrix is 2-D numpy array
    new_matrix = np.dot(embedding_matrix, vector)
    nearest_rows  = gensim.matutils.argsort(new_matrix, topn = 10, reverse = True)
    result = [(ix_to_token[row_id], float(new_matrix[row_id])) for row_id in nearest_rows]
    return result

In [ ]:
she_vector = vector_rejector(get_vector('she'),get_vector('he'))

In [ ]:
# Inspect
she_vector

In [ ]:
nearest_words(she_vector)